# Kerchunk and Xarray-Datatree

## Overview

In this tutorial we are going to use a large collection of pre-generated `Kerchunk` reference files and open them with [xarray-datatree](https://xarray-datatree.readthedocs.io/en/latest/). This chapter is heavily inspired by [this blog post](https://medium.com/pangeo/easy-ipcc-part-1-multi-model-datatree-469b87cf9114).



### About the Dataset

This collection of reference files were generated from the [**NASA NEX-GDDP-CMIP6 (Global Daily Downscaled Projections)**](https://www.nccs.nasa.gov/services/data-collections/land-based-products/nex-gddp-cmip6) dataset.  A version of this dataset is hosted on `s3` as a collection of [NetCDF files](https://registry.opendata.aws/nex-gddp-cmip6/). 


## Prerequisites
| Concepts | Importance | Notes |
| --- | --- | --- |
| [Kerchunk Basics](../foundations/kerchunk_basics) | Required | Core |
| [Multiple Files and Kerchunk](../foundations/kerchunk_multi_file) | Required | Core |
| [Kerchunk and Dask](../foundations/kerchunk_dask) | Required | Core |
| [Multi-File Datasets with Kerchunk](../case_studies/ARG_Weather.ipynb) | Required | IO/Visualization |
| [Xarray-Datatree Overview](https://xarray-datatree.readthedocs.io/en/latest/quick-overview.html)| Required | IO |

- **Time to learn**: 30 minutes

## Motivation

In total the dataset is roughly **12TB** in compressed blob storage, with a single `NetCDF` file per yearly timestep, per variable. Downloading this entire dataset for analysis on a local machine would difficult to say the least. The collection of `Kerchunk` reference files for this entire dataset is only **272 Mb**, which is about 42,000 times smaller! 

## Imports

In [ ]:
import dask
import hvplot.xarray  # noqa
import pandas as pd
import xarray as xr
from datatree import DataTree
from distributed import Client
from fsspec.implementations.reference import ReferenceFileSystem

## Read the reference catalog

The **NASA NEX-GDDP-CMIP6** dataset is organized by GCM, Scenario and Ensemble Member. Each of these Scenario/GCM combinations is represented as a combined reference file, which was created by merging across variables and concatenating along time-steps. All of these references are organized into a simple `.csv` catalog in the schema: 
|   GCM/Scenario  |      url    |
| --------------- | ----------- |


# Organzing with Xarray-Datatree
Not all of the GCM/Scenario reference datasets have shared spatial coordinates and many of the have slight differences in their calendar and thus time dimension. 
Because of this, these cannot be combined into a single `Xarray-Dataset`. Fortunately `Xarray-Datatree` provides a higher level abstraction where related `Xarray-Datasets` are organized into a tree structure where each dataset corresponds to a `leaf`.

In [ ]:
# Read the reference catalog into a Pandas DataFrame
cat_df = pd.read_csv(
    "s3://carbonplan-share/nasa-nex-reference/reference_catalog_nested.csv"
)
# Convert the DataFrame into a dictionary
catalog = cat_df.set_index("ID").T.to_dict("records")[0]

## Load Reference Datasets into Xarray-DataTree

In the following cell we create a function `load_ref_ds`, which can be parallelized via `Dask` to load `Kerchunk` references into a dictionary of `Xarray-Datasets`. 

In [ ]:
def load_ref_ds(url: str):
    fs = ReferenceFileSystem(
        url,
        remote_protocol="s3",
        target_protocol="s3",
        remote_options={"anon": True},
        target_options={"anon": True},
        lazy=True,
    )
    return xr.open_dataset(
        fs.get_mapper(),
        engine="zarr",
        backend_kwargs={"consolidated": False},
        chunks={"time": 300},
    )


tasks = {id: dask.delayed(load_ref_ds)(url) for id, url in catalog.items()}

## Use Dask Distributed to load the Xarray-Datasets from Kerchunk reference files
Using `Dask`, we are loading 164 reference datasets into memory. Since they are are `Xarray` datasets the coordinates are loaded eagerly, but the underlying data is still lazy. 

In [ ]:
client = Client(n_workers=8)
client

In [ ]:
catalog_computed = dask.compute(tasks)

## Build an Xarray-Datatree from the dictionary of datasets

In [ ]:
dt = DataTree.from_dict(catalog_computed[0])

### Accessing the Datatree

A `Datatree` is a collection of related `Xarray` datasets. We can access individual datasets using `UNIX` syntax. In the cell below, we will access a single dataset from the `datatree`.


In [ ]:
dt["ACCESS-CM2/ssp585"]

# or

dt["ACCESS-CM2"]["ssp585"]

#### Convert a Datatree node to a Dataset

In [ ]:
dt["ACCESS-CM2"]["ssp585"].to_dataset()

### Operations across a Datatree
A `Datatree` contains a collection of datasets with related coordinates and variables. Using some in-built methods, we can analyze it as if it were a single dataset. Instead of looping through hundreds of `Xarray` datasets, we can apply operations across the `Datatree`. In the example below, we will lazily create a time-series.

In [ ]:
ts = dt.mean(dim=["lat", "lon"])

### Visualize a single dataset with HvPlot

In [ ]:
display(  # noqa
    dt["ACCESS-CM2/ssp585"].to_dataset().pr.hvplot("lon", "lat", rasterize=True)
)

## Shut down the Dask cluster

In [ ]:
client.shutdown()